In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [ ]:
# Directorio de imágenes
base_dir = 'Images'  # Directorio base donde están los folders de clases

# Crear generadores para los conjuntos de entrenamiento, validación y prueba
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)  # 20% para validación dentro del entrenamiento
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generador para el conjunto de entrenamiento
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),  # Ajusta el tamaño de la imagen aquí
    batch_size=32,
    class_mode='binary',  # Para clasificación binaria
    subset='training',
    shuffle=True
)

# Generador para el conjunto de validación
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

# Generador para el conjunto de prueba
test_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Suele ser mejor no barajar para evaluaciones precisas
)


Found 3937 images belonging to 2 classes.
Found 983 images belonging to 2 classes.
Found 4920 images belonging to 2 classes.


In [ ]:
def make_classifier_model():  # Por ejemplo, para clasificar dígitos (0-9)
    model = tf.keras.Sequential()
    
    # Primera capa convolucional
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=[256, 256, 3]))  # Ajustar input_shape según el tamaño y canales de tus imágenes
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Segunda capa convolucional
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # tercera capa convolucional
    model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Aplanado de la salida
    model.add(layers.Flatten())
    # Capa de salida binaria con activación sigmoid
    model.add(layers.Dense(1, activation='sigmoid'))  # Usamos 'sigmoid' para salida binaria

    return model

In [21]:
# Crear el modelo con el número de clases deseado
model = make_classifier_model()

# Compilación del modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',   # para etiquetas enteras (p. ej., 0, 1, 2...)
              metrics=['accuracy'])

# Mostrar un resumen del modelo
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 64, 64, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 32)     │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │        32,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 341,921 (1.30 MB)

 Trainable params: 341,921 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Ajusta el número de épocas según sea necesario
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 60s 481ms/step - accuracy: 0.6592 - loss: 0.5711 - val_accuracy: 0.9948 - val_loss: 0.0157
Epoch 2/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 821us/step - accuracy: 1.0000 - loss: 0.0163 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 3/10


2024-10-30 18:39:42.130280: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


123/123 ━━━━━━━━━━━━━━━━━━━━ 60s 482ms/step - accuracy: 0.9874 - loss: 0.0333 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 4/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 1.0000 - loss: 0.0126 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 5/10


2024-10-30 18:40:42.507316: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


123/123 ━━━━━━━━━━━━━━━━━━━━ 62s 499ms/step - accuracy: 0.9965 - loss: 0.0093 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 6/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 1.0000 - loss: 1.4082e-04 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 7/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 61s 493ms/step - accuracy: 0.9989 - loss: 0.0056 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 8/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 781us/step - accuracy: 1.0000 - loss: 1.7046e-04 - val_accuracy: 1.0000 - val_loss: 1.5398e-05
Epoch 9/10


2024-10-30 18:42:46.749861: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


123/123 ━━━━━━━━━━━━━━━━━━━━ 62s 497ms/step - accuracy: 0.9966 - loss: 0.0050 - val_accuracy: 1.0000 - val_loss: 7.3074e-04
Epoch 10/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 1.0000 - val_loss: 1.4152e-04


In [23]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

153/153 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - accuracy: 0.9993 - loss: 0.0021
Test Accuracy: 99.96%
Test Loss: 0.0016


In [ ]:
# Load and preprocess the image
img_path = 'capturedImages/capture_20241030_184814.jpg'
img = image.load_img(img_path, target_size=(256, 256))  # Adjust target_size if needed
img_array = image.img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Prediction: Glasses")
else:
    print("Prediction: No Glasses")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Prediction: Glasses
